In [1]:
import yaml
import nbimporter
from datetime import datetime, date
# import pyspark
# from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType,  DateType, TimestampType
# from pyspark.sql.functions import to_date, to_timestamp
# from lab_spark import create_spark_session
from lab_schema_manager import SchemaManager
import yfinance as yf


class RawYFinance:
    
    def __init__(self, equity_type, symbol_date_pairs, object_schema_config_file, object_type, object_name):
        self.equity_type=equity_type
        self.symbol_date_pairs=symbol_date_pairs
        schema_manager=SchemaManager(object_schema_config_file)
        self.column_list=schema_manager.get_column_list(object_type, object_name)

        
    def fetch_yfinance_record(self):
        try:
            symbol, start_date = self.symbol_date_pairs
            # Fetch stock data using yfinance
            quote = yf.Ticker(symbol)
            current_date = date.today()
            hist = quote.history(start=start_date, end=current_date)
    
            # Reset index to include Date as a column and format it
            hist.reset_index(inplace=True)
            hist["Date"] = hist["Date"].dt.date

            if self.equity_type == 'stock':
                # limit and stablize the fields of hist
                hist = hist[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']]
                hist = hist[self.column_list]
                
            # Add symbol and import_time to each row
            record_list = [
                tuple(row) + (symbol, import_time) for row in hist.itertuples(index=False)
            ]
    
            # print(record_list)
            return record_list
    
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return []  # Return an empty list on error

            
    # def parallel_fetch_yfinance_record(self):
    #     try:
    #         # Distribute (symbol, start_date) pairs across Spark workers
    #         param_rdd = self.spark.sparkContext.parallelize(self.symbol_date_pairs)    
            
    #         # Fetch data in parallel
    #         mapped_record_rdd = param_rdd.flatMap(self.fetch_yfinance_record)
    
    #         # Convert RDD to DataFrame
    #         result_df = self.spark.createDataFrame(mapped_record_rdd, schema=self.column_list)
    #         # result_df = spark.createDataFrame(mapped_record_rdd)
    
    #         # Show or save the results
    #         # result_df.show()
    #         return result_df
    #     except Exception as e:
    #         print(f"Error paralleling fetch: {e}")
    #         return self.spark.createDataFrame([]) 